In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import tensorflow as tf

USE_GPU = True

dtype = torch.float64 # we will be using float throughout this tutorial
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('cpu')
print('using device:', device)

using device: cuda


## American Option Pricing with Multiple Neural Networks (method 1) article [1]
Here I'll try a simple implementation of the method I of the first article :

Here we have constant interest rate so the discount factor is $\exp(-rT)$, and the stock dynamics are modelled with Geometric Brownian Motion (GBM).

$\large dS_t = rS_tdt+\sigma S_tdW_t$

Let's simulate this GBM process by simulating variables of the natural logarithm process of the stock price $x_t = ln(S_t)$, which is normally distributed. For the dynamics of the natural logarithm process of stock prices under GBM model you need to use Ito's calculus.
$\large dx_t = \nu dt+\sigma dW_t ,  \nu = r - \frac{1}{2} \sigma ^ 2$

We can then discretize the Stochastic Differential Equation (SDE) by changing the infinitesimals $dx, dt, dz$ into small steps $\Delta x, \Delta t, \Delta W$.

$\large \Delta x = \nu  \Delta t+\sigma \Delta W$

This is the exact solution to the SDE and involves no approximation.

$\large x_{t+\Delta t} = x_{t} + \nu (\Delta t)+\sigma (W_{t+\Delta t}- W_t)$

In terms of the stock price S, we have:

$\large S_{t+\Delta t} = S_{t} \exp( \nu \Delta t + \sigma (W_{t+\Delta t}- W_t) )$

Where $(W_{t+\Delta t}- W_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$


\\


In [ ]:
def pay_off(S,r,K,dt):
  return torch.maximum(K- S,torch.zeros_like(S))

def simulate_paths(M, T, n, r, vol, S, K, dt):
    nudt = (r - 0.5 * vol**2) * dt
    lnS = np.log(S)

    # Méthode de Monte Carlo
    Z = np.random.normal(size=(M,n))
    dW=np.sqrt(dt) * Z  #it's the simulation of the dWt_i we'll need in each iteration
    delta_lnSt = nudt + vol*dW

    LnS_s = np.zeros([M, n + 1])

    # Set the initial values
    LnS_s[:, 0] = lnS
    # Compute cumulative sums using cumsum
    LnS_s[:, 1:] = np.cumsum(delta_lnSt, axis=1)
    LnS_s[:,1:] +=lnS


    S = np.exp(LnS_s)
    S_tensor = torch.tensor(S, device = device ,dtype=dtype)
    dW_tensor = torch.tensor(dW, device = device ,dtype=dtype)
    return S_tensor, dW_tensor

#Parametres

T = 1
n=50
dt = T/n  #les t_i seront donc les i*dt.


S = 36          # Prix de l'action
K = 40           # Prix d'exercice
vol = 0.2       # Volatilité (%)
r = 0.06            # Taux sans risque (%)
M = 100000 # Nombre de simulations


S,dW=simulate_paths(M, T, n, r, vol, S, K, dt)
X=torch.zeros([M,n+1], device = device ,dtype=dtype)
Y=torch.zeros_like(X, device = device ,dtype=dtype)


X[ :, n]=pay_off(S[:,n],r,K,dt)
Y[ :, n]=X[ :, n]

beta_dt=math.exp(-r*dt)



# The algorithm :

***Algorithm 1 :*** American Option Pricing with Multiple Neural Networks

**Result :** Functions $\Phi_{t_i}, \Psi_{t_i}$ for $i \in \{0,1, \ldots, n-1\}$

Simulate $N$ stock paths

Initialize $Y_{t_n}=X_{t_n}=f\left(S_{t_n}\right)$

for $i=n-1: 1$ do \\
&nbsp;&nbsp;&nbsp;&nbsp; Regress $\beta_{\Delta t}^{-1} Y_{t_{i+1}}$ on $S_{t_i}:$ \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\min_{\Phi_{t_i}, \Psi_{t_i}}\left(\beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Phi_{t_i}\left(S_{t_i}\right) - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}\right)^2$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = \beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = \beta_{\Delta t}^{-1} X_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > \Phi_{t_i}\left(S_{t_i}\right)$ then \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = f\left(S_{t_i}\right)$ \\
&nbsp;&nbsp;&nbsp;&nbsp; end \\
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > X_{t_i}$ then \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = f\left(S_{t_i}\right)$ \\
&nbsp;&nbsp;&nbsp;&nbsp; end \\
end

Regress $\beta_{\Delta t}^{-1} Y_{t_1}$ on $S_{t_0}:$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $\min \left(\beta_{\Delta t}^{-1} Y_{t_1} - \Phi_{t_0}\left(S_{t_0}\right) - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}\right)^2$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_0} = \beta_{\Delta t}^{-1} Y_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_0} = \beta_{\Delta t}^{-1} X_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class SpecificModel(nn.Module):
    def __init__(self):
        super(SpecificModel, self).__init__()
        self.branch1 = nn.Sequential(
            nn.Linear(1, 50),
            # nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(50, 50),
            # nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )
        self.branch2 = nn.Sequential(
            nn.Linear(1, 50),
            # nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(50, 50),
            # nn.BatchNorm1d(30),
            nn.ReLU(),
            nn.Linear(50, 50),
            # nn.BatchNorm1d(30),
            nn.ReLU(),
            nn.Linear(50, 1)
        )
        self.optimizer = optim.Adam(self.parameters(), lr=0.01)
        # self.optimizer = optim.SGD(self.parameters(), lr=0.01), momentum=0.9)

    def forward(self, x):
        x_branch1 = self.branch1(x)
        y_branch2 = self.branch2(x)
        concatenated_output = torch.cat((x_branch1, y_branch2), dim=1)
        return concatenated_output

    def custom_loss(self, Y_true, dW_true, y_pred):
        return torch.mean(torch.square(Y_true - (y_pred[:, 0] + dW_true * y_pred[:, 1])))

    def train_model(self, S_tensor, Y_tensor, dW_tensor, epochs=5, batch_size=32, validation_split=0.2):
        torch.cuda.empty_cache()
        combined_labels = torch.cat((Y_tensor.unsqueeze(1), dW_tensor.unsqueeze(1)), dim=1)
        dataset = TensorDataset(S_tensor, combined_labels)
        train_size = int((1 - validation_split) * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

        for epoch in range(epochs):
            self.train()
            total_loss = 0
            for S_batch, labels_batch in train_dataloader:
                self.optimizer.zero_grad()
                outputs = self(S_batch)
                loss = self.custom_loss(labels_batch[:, 0], labels_batch[:, 1], outputs)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()


            val_loss = 0

            if epoch % 3 == 0:
                self.eval()
                with torch.no_grad():
                    for S_batch, labels_batch in val_dataloader:
                        outputs = self(S_batch)
                        val_loss += self.custom_loss(labels_batch[:, 0], labels_batch[:, 1], outputs).item()

                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(train_dataset)}, Validation Loss: {val_loss/val_size}")


        self.eval()
        with torch.no_grad():
              outputs = self(S_tensor)
        print("Training complete .")

        return outputs

model = SpecificModel().to(device = device ,dtype=dtype)


In [ ]:
# The main loop :

for i in range(n-1,0,-1):
  #determination of phi and psi in one step
  # Convert data to PyTorch tensors
  # inputs = torch.tensor(S[:,i].reshape(-1,1),device = device ,dtype=dtype)
  # scaler = StandardScaler()
  # normalized_S = scaler.fit_transform(S[:,i].reshape(-1,1))
  print( "In the ",i ," th step :")
  outputs =  model.train_model(S[:,i].reshape(-1,1), beta_dt*Y[:,i+1].reshape(-1,1), dW[:,i].reshape(-1,1))

  psi_S=outputs[:,1]
  phi_S=outputs[:,0]

  X[ :, i]=beta_dt*X[:,i+1]-psi_S *dW[:,i] #to change it into (M,)
  Y[ :, i]=beta_dt*Y[:,i+1]-psi_S *dW[:,i]

  Z=pay_off(S[:,i],r,K,dt)

  Y[ :, i] = torch.where(Z> phi_S, Z, Y[ :, i])
  X[ :, i] = torch.where(Z> X[:,i], Z, X[ :, i])


#Pour i =0 :
#determination of phi and psi in two steps
i=0
outputs = model.train_model(S[:,i].reshape(-1,1), beta_dt*Y[:,i+1].reshape(-1,1), dW[:,i].reshape(-1,1))

psi_S=outputs[:,1]
phi_S=outputs[:,0]

X[ :, 0]=beta_dt*X[:,1]-psi_S *dW[:,0]
Y[ :, 0]=beta_dt*Y[:,1]-psi_S *dW[:,0]




In [ ]:
#Monté Carlo
u0=torch.mean(X[:,0]).cpu().numpy()
l0=torch.mean(Y[:,0]).cpu().numpy()
print(u0)
print(l0)


6.859674071781852
4.704045255605416


With the same parameters and network structure  [(20,20),(20,20,20)] of page 17 in  article [1] , the results are :

The upper bound we got is 4.565703035603622.

The lower bound we got 4.437699059733613 .

Article [1] showed that , in this case, the lower bound exhibits fluctuations with a mean of 4.4762, while the upper bound shows fluctuations with a mean of
4.4887.


We needed 2 hours to get this result!!


In [ ]:
print(np.abs(u0-l0))
print(4.4887-4.4762)
print(np.abs(np.abs(u0-l0)-(4.4887-4.4762)))

2.155628816176436
0.01249999999999929
2.1431288161764366


The difference (0.128 ) is what we wanted (0.0125 ) multiplied by 10, that's why we have to improve our code to get better results and in less time !!